In [1]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

import wavy
from wavy.models import compute_default_scores
from wavy.utils import reverse_pct_change

/Users/rodrigonader/anaconda3/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol

In [4]:
msft = yf.Ticker("MSFT")

In [5]:
# get historical market data from 2005 on
hist = msft.history(period="max", start="2005-01-01")

In [6]:
hist[['Open', 'High', 'Low', 'Close']].plot()

In [7]:
hist = hist[['Open', 'High', 'Low', 'Close']]

In [8]:
df = hist.pct_change()

In [9]:
df.plot()

In [10]:
print(len(df) - len(df.dropna()))
df.dropna(inplace=True)

1


In [11]:
df.High.hist(bins=200)

In [12]:
df = df[['Close']]

In [13]:
x, y = wavy.create_panels(df, lookback=1, gap=0, horizon=1)

In [14]:
x.plot()

In [15]:
y.values

array([[[-0.00223548]],

       [[-0.00112035]],

       [[-0.0029904 ]],

       ...,

       [[-0.02078324]],

       [[-0.04456363]],

       [[-0.04241279]]])

In [16]:
baseline = wavy.models.BaselineConstant(x, y, model_type="regression", constant=0)

baseline.fit()
baseline.score()

99/99 [==============================] - 0s 908us/step - loss: 0.0000e+00 - MAE: 0.0110 - val_loss: 0.0000e+00 - val_MAE: 0.0128


2022-06-13 20:16:40.456625: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


,train,test,val
loss,0.000000,0.000000,0.000000
MAE,0.011032,0.012612,0.012753


In [17]:
shift = wavy.models.BaselineShift(x, y, model_type="regression")

shift.fit()
shift.score()

99/99 [==============================] - 0s 2ms/step - loss: 5.8656e-04 - MAE: 0.0163 - val_loss: 0.0010 - val_MAE: 0.0199


,train,test,val
loss,0.000587,0.000586,0.001012
MAE,0.016267,0.018841,0.019854


In [18]:
lr = wavy.models.LinearRegression(x, y)
lr.fit(epochs=30, verbose=False)
lr.score()

,train,test,val
loss,0.000276,0.000275,0.000369
MAE,0.011039,0.012607,0.012511


In [19]:
dense = wavy.models.DenseModel(x, y, 'regression', dense_layers=3)
dense.fit(epochs=30, verbose=False)
dense.score()

,train,test,val
loss,0.000276,0.000277,0.000373
MAE,0.011039,0.012626,0.012588


In [31]:
yabv = y > 0
yabv = yabv.astype(int)

In [36]:
log = wavy.models.LogisticRegression(x, yabv)
log.fit(epochs=200, verbose=True)
log.score()

Epoch 1/200
99/99 [==============================] - 0s 2ms/step - loss: 0.6929 - auc: 0.5192 - accuracy: 0.5146 - val_loss: 0.6918 - val_auc: 0.5668 - val_accuracy: 0.5588
Epoch 2/200
99/99 [==============================] - 0s 603us/step - loss: 0.6929 - auc: 0.5169 - accuracy: 0.5165 - val_loss: 0.6919 - val_auc: 0.5655 - val_accuracy: 0.5461
Epoch 3/200
99/99 [==============================] - 0s 594us/step - loss: 0.6929 - auc: 0.5211 - accuracy: 0.5032 - val_loss: 0.6920 - val_auc: 0.5769 - val_accuracy: 0.5272
Epoch 4/200
99/99 [==============================] - 0s 532us/step - loss: 0.6929 - auc: 0.5123 - accuracy: 0.5130 - val_loss: 0.6917 - val_auc: 0.5678 - val_accuracy: 0.5525
Epoch 5/200
99/99 [==============================] - 0s 538us/step - loss: 0.6929 - auc: 0.5172 - accuracy: 0.5136 - val_loss: 0.6918 - val_auc: 0.5747 - val_accuracy: 0.5487
Epoch 6/200
99/99 [==============================] - 0s 608us/step - loss: 0.6929 - auc: 0.5167 - accuracy: 0.5076 - val_loss: 

,train,test,val
loss,0.692383,0.693029,0.689273
auc,0.521225,0.521185,0.576191
accuracy,0.513291,0.526196,0.543616


In [ ]:
x, y = wavy.create_panels(df, lookback=120, gap=0, horizon=30)

In [ ]:
x[0]

In [ ]:
ymax = y.max() > 0.01

In [ ]:
ymax[0]

In [ ]:
ymax.as_dataframe()

In [ ]:
lr = wavy.models.LinearRegression(x, y)
lr.fit(epochs=30, verbose=False)
lr.score()

In [ ]:
dense = wavy.models.DenseModel(x, y, 'regression')
dense.fit(epochs=3, verbose=True)
dense.score()

In [ ]:
conv = wavy.models.ConvModel(x, y, 'regression')
conv.fit(epochs=3, verbose=True)
conv.score()

In [ ]:
model.predict(x.values)

In [ ]:
y_pred = baseline.predict()

In [ ]:
y.plot()

In [ ]:
y_pred.plot()

In [ ]:
rev_y = reverse_pct_change(y, hist)

In [ ]:
rev_y.plot()

In [ ]:
rev_y_pred = reverse_pct_change(y_pred, hist)

In [ ]:
rev_y_pred.plot()

In [ ]:
# How to interpret that error? What does it mean comparing to the mean/std of the val set?
np.median(baseline.y_val.flatten())

In [ ]:
residuals = baseline.residuals()

In [ ]:
residuals.plot() # Highest error is in train set

In [ ]:
compute_default_scores(x_high, y, 'regression', ['MAE', 'MSE'])

In [ ]:
conv = wavy.ConvModel(x_high, y, model_type="regression", conv_layers=1, conv_filters=32,kernel_size=3, dense_layers=1, dense_units=32)
conv.fit(epochs=20)

# using a cannon ball to kill an ant!

In [ ]:
params=[{'conv_layers': [1, 5, 10], 'conv_filters': [16, 32], 'kernel_size': [2, 3, 7], 'dense_layers': [1], 'dense_units': [32]}, {'dense_layers': 1, 'dense_units': 32}]

models = [ConvModel, DenseModel]

# wavy.GridSearchModel(x, y, model_type, models=, )

In [ ]:
lr.score()

In [ ]:
grid_search --> sklearn

In [ ]:
conv.plot_residuals()

In [ ]:
# TODO: Compare residuals between models

In [ ]:
default_scores = compute_default_scores(x, y, model_type="regression", metrics=["MAE", "MSE"])

In [ ]:
# rev_y = reverse_pct_change(y, hist).as_dataframe()
# rev_y_conv = reverse_pct_change(conv.predict(), hist).as_dataframe()
# rev_y_base = reverse_pct_change(baseline.predict(), hist).as_dataframe()

# rev_y_conv.columns = ['conv_' + col for col in rev_y_conv.columns]
# rev_y_base.columns = ['base_' + col for col in rev_y_base.columns]

In [ ]:
# plot_dataframes([rev_y, rev_y_conv, rev_y_base])

In [ ]:
y_pred[0]

In [ ]:
hist.iloc[10:20]

In [ ]:
y_orig.as_dataframe()

In [ ]:
y.as_dataframe()

In [ ]:
y_orig = y.update(hist.shift())

In [ ]:
y_orig[0]

In [ ]:
y_pred_orig = y_pred.shift() + y_orig

In [ ]:
y_pred_orig.plot()

In [ ]:
y_true = y.test.as_dataframe()
y_baseline = baseline.predict().as_dataframe()
y_lr = lr.predict().as_dataframe()
y_dense = dense.predict().as_dataframe()
y_conv = conv.predict().as_dataframe()

y_true.columns = ['true_' + col for col in y_true.columns]
y_baseline.columns = ['baseline_' + col for col in y_baseline.columns]
y_lr.columns = ['lr_' + col for col in y_lr.columns]
y_dense.columns = ['dense_' + col for col in y_dense.columns]
y_conv.columns = ['conv_' + col for col in y_conv.columns]

plot_dataframes([y_true, y_baseline, y_lr, y_dense, y_conv])

In [ ]:
baseline

In [ ]:
dummy = wavy.ShallowModel(x_high, y, model=DummyRegressor, metrics=[mean_absolute_error], strategy="mean")
dummy.fit()

In [ ]:
rf = wavy.ShallowModel(x, y, model=RandomForestRegressor, metrics=[mean_absolute_error], n_estimators=50)
rf.fit()

In [ ]:
dense = wavy.DenseModel(x, y, model_type="regression")
dense.fit(epochs=20)

In [ ]:
# Try the same models but with all input columns
# TODO: Use function to compute default results per model

# lr = wavy.LinearRegression(x, y)
# lr.fit(epochs=20)

In [ ]:

# data1 = {'train': x.train.as_dataframe()["High"],
#         'val': x.val.as_dataframe()["High"],
#         'test': x.test.as_dataframe()["High"]
#         }

# data2 = {'train': x.train.as_dataframe()["Low"],
#         'val': x.val.as_dataframe()["Low"],
#         'test': x.test.as_dataframe()["Low"]
#         }

In [ ]:
aaa = y[-1000:]

s1 = y.train.as_dataframe()['High']
s2 = y.val.as_dataframe()['High']
s3 = y.test.as_dataframe()['High']

In [ ]:
fig = Figure()

In [ ]:
# fig.linebar(s)
# fig.scatter(s)

fig.dotbar(s1)
fig.dotbar(s2, color="blue")
fig.dotbar(s3)

In [ ]:
fig.fig

In [ ]:
fig = PanelFigure()
fig.split_sets(data1)
fig.add_area(data1)
fig.add_dotbar(data2)

In [ ]:
fig.compile()